In [1]:
import torch
print(f"torch:{torch.__version__}\({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

torch:1.9.1+cu111\(NVIDIA GeForce RTX 3060)


c:\Users\admin\anaconda3\envs\pre_pjt\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
pip install beautifulsoup4

  Using cached beautifulsoup4-4.12.2-py3-none-any.whl (142 kB)
  Obtaining dependency information for soupsieve>1.2 from https://files.pythonhosted.org/packages/4c/f3/038b302fdfbe3be7da016777069f26ceefe11a681055ea1f7817546508e3/soupsieve-2.5-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [27]:
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from bs4 import BeautifulSoup
import matplotlib.patches as patches
from PIL import Image
import random
import cv2
import time
import pandas as pd
from collections import Counter
import torch
import numpy as np
import shutil
import torchvision
from torch.utils.data import Dataset
from torchvision import transforms

In [29]:
img_lst = glob.glob('C:/Users/admin/Desktop/data_V4/images/*.jpg')
annot_lst = glob.glob('C:/Users/admin/Desktop/data_V4/annotations/*.xml')
print(len(img_lst))
print(len(annot_lst))

4847
4847


In [30]:
os.remove('C:/Users/admin/Desktop/data_V4/images/desktop.ini')

In [31]:
os.mkdir ('C:/Users/admin/Desktop/data_V4/test_images')
os.mkdir ('C:/Users/admin/Desktop/data_V4/test_annotations')

In [32]:
import random
random.seed(1234)
idx = random.sample(range(641), 170)
print(len(idx))
print(idx[:10])

170
[451, 119, 7, 92, 596, 35, 85, 100, 363, 242]


In [33]:
import numpy as np
import shutil

for img in np.array(sorted(os.listdir('C:/Users/admin/Desktop/data_V4/images')))[idx]:
    shutil.move('C:/Users/admin/Desktop/data_V4/images/'+img, 'C:/Users/admin/Desktop/data_V4/test_images/'+img)

for annot in np.array(sorted(os.listdir('C:/Users/admin/Desktop/data_V4/annotations')))[idx]:
    shutil.move('C:/Users/admin/Desktop/data_V4/annotations/'+annot, 'C:/Users/admin/Desktop/data_V4/test_annotations/'+annot)

In [34]:
print(len(os.listdir('C:/Users/admin/Desktop/data_V4/annotations')))
print(len(os.listdir('C:/Users/admin/Desktop/data_V4/images')))
print(len(os.listdir('C:/Users/admin/Desktop/data_V4/test_annotations')))
print(len(os.listdir('C:/Users/admin/Desktop/data_V4/test_images')))

4677
4677
170
170


In [35]:
def generate_box(obj):

    xmin = float(obj.find('xmin').text)
    ymin = float(obj.find('ymin').text)
    xmax = float(obj.find('xmax').text)
    ymax = float(obj.find('ymax').text)

    return [xmin, ymin, xmax, ymax]


def generate_label(obj):
    return 0


def generate_target(file):
    with open(file) as f:
        data = f.read()
        soup = BeautifulSoup(data, "html.parser")
        objects = soup.find_all("object")

        num_objs = len(objects)

        boxes = []
        labels = []
        for i in objects:
            boxes.append(generate_box(i))
            labels.append(generate_label(i))

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels

        return target


def collate_fn(batch):
    return tuple(zip(*batch))

In [36]:
class Dataset_transformer(object):
    def __init__(self, transforms, path):
        self.transforms = transforms
        self.path = path
        self.imgs = list(sorted(os.listdir(self.path)))


    def __getitem__(self, idx):
        file_image = self.imgs[idx]
        file_label = self.imgs[idx][:-3] + 'xml'
        img_path = os.path.join(self.path, file_image)

        if 'test' in self.path:
            label_path = os.path.join("C:/Users/admin/Desktop/data_V4/test_annotations/", file_label)
        else:
            label_path = os.path.join("C:/Users/admin/Desktop/data_V4/annotations/", file_label)

        img = Image.open(img_path)
        target = generate_target(label_path)

        if self.transforms is not None:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [37]:
data_transform = transforms.Compose([transforms.ToTensor()])

In [38]:
dataset = Dataset_transformer(data_transform, 'C:/Users/admin/Desktop/data_V4/images/')
data_loader = torch.utils.data.DataLoader(dataset, batch_size=4, collate_fn=collate_fn)

In [39]:
test_dataset = Dataset_transformer(data_transform, 'C:/Users/admin/Desktop/data_V4/test_images/')
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=2, collate_fn=collate_fn)

In [40]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [41]:
def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

In [42]:
model = get_model(1)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to C:\Users\admin/.cache\torch\hub\checkpoints\fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:10<00:00, 15.3MB/s] 


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [43]:
torch.cuda.is_available()

True

In [47]:
num_epochs = 50
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

In [48]:
print('----------------------train start--------------------------')
for epoch in range(num_epochs):
    start = time.time()
    model.train()
    i = 1
    epoch_loss = 0
    for imgs, annotations in data_loader:
        i += 1
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        loss_dict = model(imgs, annotations)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        epoch_loss += losses
    print(f'epoch : {epoch+1}, Loss : {epoch_loss}, time : {time.time() - start}')

----------------------train start--------------------------
epoch : 1, Loss : 45.72172546386719, time : 901.9207012653351
epoch : 2, Loss : 32.95088195800781, time : 900.406046628952
epoch : 3, Loss : 30.59593963623047, time : 900.5560805797577
epoch : 4, Loss : 29.523103713989258, time : 900.2224538326263
epoch : 5, Loss : 28.996932983398438, time : 900.5072689056396
epoch : 6, Loss : 28.90983772277832, time : 899.8542602062225
epoch : 7, Loss : 28.343542098999023, time : 899.6052398681641
epoch : 8, Loss : 28.258258819580078, time : 899.7053425312042
epoch : 9, Loss : 28.2061710357666, time : 898.938038110733
epoch : 10, Loss : 27.57108497619629, time : 899.3049802780151
epoch : 11, Loss : 27.082443237304688, time : 899.1226403713226
epoch : 12, Loss : 26.527917861938477, time : 899.0871086120605
epoch : 13, Loss : 25.90780258178711, time : 898.9044630527496
epoch : 14, Loss : 25.511919021606445, time : 898.8212013244629
epoch : 15, Loss : 25.298513412475586, time : 899.1721782684326

In [49]:
torch.save(model.state_dict(),f'C:/Users/admin/Desktop/RCNN_{num_epochs}.pt')